In [1]:
import rasterio
from rasterio.merge import merge
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os, time
from pathlib import Path 
import glob
from shapely.geometry import box

In [2]:
path = Path('/Users/bjafino/Documents/GitHub/PovertyClimateFall2020/FATHOM')
coastalFlood_Folder = 'Coastal_flood'
coastalFlood_path = path / coastalFlood_Folder

search_criteria = 'inundationMap*'
q = os.path.join(coastalFlood_path, search_criteria)

coastal_fps = glob.glob(q)

In [3]:
coastal_fps

['\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_-180_0_-90_-90.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_-180_90_-90_0.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_-90_0_0_-90.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_-90_90_0_0.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_0_0_90_-90.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_0_90_90_0.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_90_0_180_-90.tif',
 '\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateFall2020\\FATHOM\\Coastal_flood\\inundationMap_90_90_180_0.tif']

In [4]:
src_files_to_mosaic = []

for fp in coastal_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

## Bruteforcing - 26 March

In [23]:
import rasterio as rs
from rasterio.plot import show
from rasterio.enums import Resampling
from rasterio.plot import plotting_extent
from rasterio.merge import merge
import rasterio.mask
from rasterio import shutil as rio_shutil
from rasterio.vrt import WarpedVRT
import shapely
from shapely.geometry.multipolygon import MultiPolygon

import geopandas as gpd

In [26]:
def export_raster_to_geotiff(array, meta, Cropped_outputfile):
    with rs.open(Cropped_outputfile, 'w', **meta, compress = 'LZW', tiled=True) as dest:
        dest.write(array)
    return Cropped_outputfile

In [27]:
gdf = gpd.read_file('GMGD_world_admin/newold_geo_code2_povdata_v7.shp')

In [34]:
all_raster = {}

for i, df in enumerate(range(len(gdf))):
    gdf_ = gdf.iloc[i]
    
    all_rasters = []
    all_trans = []
    
    fname = 'out/dir/CoastalFloodNew_{}_{}*.tif'.format(gdf_['code'],i)
    calculated = glob.glob(fname)
    
    if len(calculated) > 0:
        #print(gdf_['code'],i, 'already calculated')
        continue
        
    for dataset in src_files_to_mosaic:
        try:
            geom = box(gdf_.geometry.bounds[0], gdf_.geometry.bounds[1], gdf_.geometry.bounds[2], gdf_.geometry.bounds[3])
            
            if type(geom) != shapely.geometry.multipolygon.MultiPolygon:
                geom = MultiPolygon([geom])
                
            out_array, out_trans  = rs.mask.mask(dataset=dataset, shapes=geom, 
                                                 crop=True, nodata=0, filled=False, all_touched=True)
            all_rasters.append(out_array)
            all_trans.append(out_trans)
            
        except:
            pass
    
    try:
    
        if len(all_rasters)==1:
            out_trans = all_trans[0]
            out_raster = all_rasters[0]
            out_meta = dataset.meta.copy()
            out_meta.update({"height": out_raster.shape[1], 
                             "width": out_raster.shape[2],
                             "transform": out_trans,
                             "nodata":0})

            export_raster_to_geotiff(out_raster, out_meta,
                                     'out/dir/CoastalFloodNew_{}_{}.tif'.format(gdf_['code'],i))

            print(gdf_['code'], i, 'Saved')

        elif len(all_rasters)>1:
            for r, out_raster in enumerate(all_rasters):
                out_trans = all_trans[r]
                out_raster = all_rasters[r]
                out_meta = dataset.meta.copy()
                out_meta.update({"height": out_raster.shape[1], 
                                 "width": out_raster.shape[2],
                                 "transform": out_trans,
                                 "nodata":0})

                export_raster_to_geotiff(out_raster, out_meta,
                                         'out/dir/CoastalFloodNew_{}_{}_{}.tif'.format(gdf_['code'],i,r))
            print(gdf_['code'],i, 'Saved')
            
        elif len(all_rasters)==0:
            print(gdf_['code'], gdf_['geo_code2'], i, 'not processed, no coastal risk')
    
    except:
        print(gdf_['code'], i, 'failed saving')

TWN 4 Saved
TWN 12 Saved
TWN 14 Saved
TWN 15 Saved
TWN 16 Saved
TWN 17 Saved
TWN 18 Saved
TWN 22 Saved
WSM 24 Saved
IDN 43 Saved
MMR 75 Saved
MNG 79 Saved
MNG 80 Saved
MNG 81 Saved
MNG 82 Saved
MNG 83 Saved
MNG 84 Saved
MNG 85 Saved
MNG 86 Saved
MNG 87 Saved
MNG 88 Saved
MNG 89 Saved
MNG 90 Saved
MNG 91 Saved
MNG 92 Saved
MNG 93 Saved
MNG 94 Saved
MNG 95 Saved
MNG 96 Saved
MNG 97 Saved
MNG 98 Saved
MNG 99 Saved
MNG 100 Saved
MYS 105 Saved
MYS 114 Saved
PNG 117 Saved
THA 120 Saved
THA 121 Saved
THA 122 Saved
THA 123 Saved
THA 124 Saved
THA 125 Saved
THA 126 Saved
THA 127 Saved
THA 128 Saved
THA 129 Saved
THA 134 Saved
THA 136 Saved
THA 137 Saved
THA 138 Saved
THA 139 Saved
THA 140 Saved
THA 141 Saved
THA 142 Saved
THA 143 Saved
THA 144 Saved
THA 145 Saved
THA 146 Saved
THA 147 Saved
THA 148 Saved
THA 149 Saved
THA 150 Saved
THA 151 Saved
THA 152 Saved
THA 153 Saved
THA 154 Saved
THA 155 Saved
THA 156 Saved
THA 157 Saved
THA 158 Saved
THA 159 Saved
THA 160 Saved
THA 161 Saved
THA 162 Sav

TJK 946 Saved
TJK 947 Saved
TJK 949 Saved
SVN 950 Saved
ESP 956 Saved
ESP 960 Saved
ESP 962 Saved
ESP 963 Saved
ESP 966 Saved
LVA 971 Saved
HUN 972 Saved
HUN 973 Saved
HUN 974 Saved
CZE 979 Saved
CZE 980 Saved
CZE 981 Saved
CZE 982 Saved
CZE 983 Saved
CZE 984 Saved
CZE 985 Saved
CZE 986 Saved
BLR 987 Saved
BLR 988 Saved
BLR 989 Saved
BLR 990 Saved
BLR 991 Saved
BLR 992 Saved
BLR 993 Saved
KAZ 997 Saved
KAZ 998 Saved
KAZ 999 Saved
KAZ 1001 Saved
KAZ 1002 Saved
KAZ 1003 Saved
KAZ 1004 Saved
KAZ 1006 Saved
KAZ 1007 Saved
KAZ 1008 Saved
KAZ 1009 Saved
KAZ 1010 Saved
KAZ 1011 Saved
KGZ 1012 Saved
KGZ 1014 Saved
KGZ 1015 Saved
KGZ 1016 Saved
KGZ 1017 Saved
KGZ 1018 Saved
KGZ 1019 Saved
KGZ 1020 Saved
TJK 1021 Saved
UKR 1022 Saved
MDA 1023 Saved
MDA 1024 Saved
MDA 1025 Saved
KAZ 1026 Saved
MDA 1027 Saved
AZE 1031 Saved
AZE 1033 Saved
AZE 1036 Saved
IND 1038 Saved
IND 1041 Saved
IND 1042 Saved
IND 1045 Saved
IND 1046 Saved
IND 1051 Saved
IND 1052 Saved
IND 1053 Saved
IND 1054 Saved
IND 1056 Sa

GIN 1787 Saved
MLI 1788 Saved
MLI 1789 Saved
MLI 1790 Saved
MLI 1791 Saved
MLI 1792 Saved
MLI 1793 Saved
MLI 1794 Saved
MLI 1795 Saved
MLI 1796 Saved
TCD 1797 Saved
TCD 1798 Saved
TCD 1799 Saved
TCD 1800 Saved
TCD 1801 Saved
TCD 1802 Saved
TCD 1803 Saved
TCD 1804 Saved
TCD 1805 Saved
TCD 1806 Saved
TCD 1807 Saved
TCD 1808 Saved
TCD 1809 Saved
TCD 1810 Saved
TCD 1811 Saved
TCD 1812 Saved
TCD 1813 Saved
TCD 1814 Saved
TCD 1815 Saved
TCD 1816 Saved
UGA 1817 Saved
UGA 1818 Saved
UGA 1819 Saved
UGA 1820 Saved
EGY 1824 Saved
DJI 1825 Saved
DJI 1826 Saved
IRQ 1831 Saved
IRQ 1833 Saved
IRQ 1834 Saved
IRQ 1835 Saved
IRQ 1836 Saved
IRQ 1837 Saved
IRQ 1838 Saved
IRQ 1839 Saved
IRQ 1840 Saved
IRQ 1841 Saved
IRQ 1842 Saved
IRQ 1843 Saved
IRQ 1844 Saved
IRQ 1845 Saved
IRQ 1846 Saved
IRQ 1847 Saved
IRQ 1848 Saved
LBN 1849 Saved
LBN 1850 Saved
LBN 1851 Saved
LBN 1852 Saved
LBN 1853 Saved
LBN 1854 Saved
TUN 1855 Saved
TUN 1860 Saved
CHN 1862 Saved
CHN 1863 Saved
CHN 1864 Saved
CHN 1866 Saved
CHN 1869 S